# Importing-Preprocessing Dataset

In [1]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split

samples = pd.read_csv('wines.csv')
samples['type'] = samples['type'].replace({'white': 0, 'red': 1}).astype(int)
samples[:] = KNNImputer(n_neighbors=25, weights='distance').fit_transform(samples)

samples['other sulfur dioxide'] = samples['total sulfur dioxide'] - samples['free sulfur dioxide']

targets = samples['quality']
samples.drop(columns=['quality', 'total sulfur dioxide'], inplace=True)

x_train, x_test, y_train, y_test = train_test_split(samples, targets, test_size=0.1, random_state=0)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((5847, 12), (5847,), (650, 12), (650,))

# Define Scaling Pipelines

In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer

scaling_methods = {
    'None': None,
    'MinMax': MinMaxScaler(),
    'StandardScaler': StandardScaler(),
    'RobustScaler': RobustScaler(),
    'Normalizer': Normalizer()
}

# Training a Neural Network

In [3]:
import tensorflow as tf
import tensorflow_addons as tfa

tf.random.set_seed(0)


def build_model(input_size):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(input_size))
    model.add(tf.keras.layers.Dense(units=256, activation='gelu', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(units=128, activation='gelu'))
    model.add(tf.keras.layers.Dense(1))

    model.compile(optimizer=tfa.optimizers.Yogi(learning_rate=0.0005), loss='mae', metrics=['mae', 'mape'])

    return model


model = build_model(samples.shape[1:])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               3072      
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 37,121
Trainable params: 36,609
Non-trainable params: 512
_________________________________________________________________


In [4]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

epochs = 300
batch_size = 32
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=30
)

for scaling_method, scaler in scaling_methods.items():
    if scaler is not None:
        scaled_x_train = scaler.fit_transform(x_train)
        scaled_x_test = scaler.transform(x_test)
    else:
        scaled_x_train = x_train
        scaled_x_test = x_test

    model = build_model(x_train.shape[1:])
    model.fit(
        scaled_x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        callbacks=[early_stopping],
        validation_data=(scaled_x_test, y_test),
        verbose=0
    )
    y_pred = model.predict(scaled_x_test)

    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    print(f'MAE: {mae}\tMAPE: {mape}\tScaling Method: {scaling_method}')

21/21 [==============================] - 0s 848us/step
MAE: 0.574664941567641	MAPE: 0.10425148522548187	Scaling Method: None
21/21 [==============================] - 0s 798us/step
MAE: 0.5335880668346699	MAPE: 0.09286738122164548	Scaling Method: MinMax
21/21 [==============================] - 0s 798us/step
MAE: 0.5073002829918495	MAPE: 0.08849438374645108	Scaling Method: StandardScaler
21/21 [==============================] - 0s 798us/step
MAE: 0.5373223462471595	MAPE: 0.09617116946877141	Scaling Method: RobustScaler
21/21 [==============================] - 0s 848us/step
MAE: 0.6214593729606042	MAPE: 0.11294098635645576	Scaling Method: Normalizer


| Scaling        | Mean Absolute Error **(MAE)** | Mean Absolute Percentage Error **(MAPE)** |
|----------------|-------------------------------|-------------------------------------------|
| No Scaling     | 0.5747                        | 0.1043                                    |
| MinMax         | 0.5336                        | 0.0923                                    |
| StandardScaler | **0.5073**                    | **0.0885**                                |
| RobustScaler   | 0.5373                        | 0.0962                                    |
| Normalizer     | 0.6215                        | 0.1129                                    |

# Improving the Regression Model

Obviously, the red wine has different properties than the white wine. We can use this to scale differently
the 2 types of wine.

In [5]:
white_wine_ind = samples['type'] == 0.0
red_wine_ind = samples['type'] == 1.0

for scaling_method in scaling_methods:
    if scaling_method != 'None':
        scaled_x_train = x_train.copy()
        scaled_x_test = x_test.copy()

        white_scaler = scaling_methods[scaling_method]
        scaled_x_train.loc[white_wine_ind] = white_scaler.fit_transform(scaled_x_train.loc[white_wine_ind])
        scaled_x_test.loc[white_wine_ind] = white_scaler.transform(scaled_x_test.loc[white_wine_ind])

        red_scaler = scaling_methods[scaling_method]
        scaled_x_train.loc[red_wine_ind] = red_scaler.fit_transform(scaled_x_train.loc[red_wine_ind])
        scaled_x_test.loc[red_wine_ind] = red_scaler.transform(scaled_x_test.loc[red_wine_ind])

        model = build_model(x_train.shape[1:])
        model.fit(
            scaled_x_train,
            y_train,
            batch_size=batch_size,
            epochs=epochs,
            callbacks=[early_stopping],
            validation_data=(scaled_x_test, y_test),
            verbose=0
        )
        y_pred = model.predict(scaled_x_test)

        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)

        print(f'MAE: {mae}\tMAPE: {mape}\tScaling Method: {scaling_method}')

21/21 [==============================] - 0s 848us/step
MAE: 0.5506982950063852	MAPE: 0.09579753529021155	Scaling Method: MinMax
21/21 [==============================] - 0s 758us/step
MAE: 0.5123593665544803	MAPE: 0.09001121112475029	Scaling Method: StandardScaler
21/21 [==============================] - 0s 868us/step
MAE: 0.5049129166969886	MAPE: 0.08913842107612135	Scaling Method: RobustScaler
21/21 [==============================] - 0s 798us/step
MAE: 0.5900471496582032	MAPE: 0.10255494306201027	Scaling Method: Normalizer


| Scaling        | Mean Absolute Error **(MAE)** | Mean Absolute Percentage Error **(MAPE)** |
|----------------|-------------------------------|-------------------------------------------|
| No Scaling     | 0.5747                        | 0.1043                                    |
| MinMax         | 0.5507                        | 0.0958                                    |
| StandardScaler | 0.5124                        | 0.0900                                    |
| RobustScaler   | **0.5049**                    | **0.0891**                                |
| Normalizer     | 0.5900                        | 0.1026                                    |

We may notice that by using separate scalers for white and red wine samples the overal MAE was reduced

# Hyper-Parameter Tuning with Optuna

In [4]:
import optuna

white_wine_ind = samples['type'] == 0.0
red_wine_ind = samples['type'] == 1.0

scaler = StandardScaler()
x_train.loc[white_wine_ind] = scaler.fit_transform(x_train.loc[white_wine_ind])
x_test.loc[white_wine_ind] = scaler.transform(x_test.loc[white_wine_ind])

scaler = StandardScaler()
x_train.loc[red_wine_ind] = scaler.fit_transform(x_train.loc[red_wine_ind])
x_test.loc[red_wine_ind] = scaler.transform(x_test.loc[red_wine_ind])

In [8]:
def create_model(trial):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(x_train.shape[1:]))

    n_layers = trial.suggest_int('n_layers', 1, 3)
    for i in range(n_layers - 1):
        hidden_units = trial.suggest_int(f'hidden_units_{i}', 32, 256)
        use_batch_norm = bool(trial.suggest_int(f'use_batch_norm_{i}', 0, 1))
        use_bias = False if use_batch_norm else True

        model.add(tf.keras.layers.Dense(units=hidden_units, activation='gelu', use_bias=use_bias))
        if use_batch_norm:
            model.add(tf.keras.layers.BatchNormalization())

    hidden_units = trial.suggest_int('hidden_units_last', 32, 256)
    model.add(tf.keras.layers.Dense(units=hidden_units, activation='gelu'))
    model.add(tf.keras.layers.Dense(units=1))

    optimizer = trial.suggest_categorical('optimizer', ['Yogi', 'AdamW'])
    loss = trial.suggest_categorical('loss', ['mse', 'mae', 'mape'])

    if optimizer == 'Adam':
        optim = tf.keras.optimizers.Adam()
    elif optimizer == 'Yogi':
        optim = tfa.optimizers.Yogi()
    else:
        optim = tfa.optimizers.AdamW(weight_decay=0.01)

    model.compile(optimizer=optim, loss=loss, metrics=[loss])

    return model


def objective(trial):
    model = create_model(trial)

    if trial.should_prune():
        raise optuna.TrialPruned()

    model.fit(
        x_train,
        y_train,
        batch_size=32,
        epochs=100,
        callbacks=[early_stopping],
        validation_data=(x_test, y_test),
        verbose=0
    )

    y_pred = model.predict(x_test)
    return mean_absolute_error(y_test, y_pred)


n_trials = 100
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)
study.trials_dataframe()

[I 2022-07-15 18:07:44,977] A new study created in memory with name: no-name-20b38cfc-5d38-4c7e-9700-94c70a8a35ef


21/21 [==============================] - 0s 748us/step


[I 2022-07-15 18:08:40,118] Trial 0 finished with value: 0.5385948093120868 and parameters: {'n_layers': 2, 'hidden_units_0': 169, 'use_batch_norm_0': 0, 'hidden_units_last': 208, 'optimizer': 'Yogi', 'loss': 'mae'}. Best is trial 0 with value: 0.5385948093120868.


21/21 [==============================] - 0s 720us/step


[I 2022-07-15 18:09:06,697] Trial 1 finished with value: 5.213332895584978 and parameters: {'n_layers': 1, 'hidden_units_last': 44, 'optimizer': 'AdamW', 'loss': 'mape'}. Best is trial 0 with value: 0.5385948093120868.


...

21/21 [==============================] - 0s 748us/step


[I 2022-07-15 19:41:12,424] Trial 98 finished with value: 0.4849989898388202 and parameters: {'n_layers': 3, 'hidden_units_0': 213, 'use_batch_norm_0': 0, 'hidden_units_1': 225, 'use_batch_norm_1': 0, 'hidden_units_last': 256, 'optimizer': 'Yogi', 'loss': 'mape'}. Best is trial 55 with value: 0.4111724328994751.


21/21 [==============================] - 0s 731us/step


[I 2022-07-15 19:41:49,738] Trial 99 finished with value: 0.5603517620380108 and parameters: {'n_layers': 1, 'hidden_units_last': 234, 'optimizer': 'Yogi', 'loss': 'mae'}. Best is trial 55 with value: 0.4111724328994751.


,number,value,datetime_start,datetime_complete,duration,params_hidden_units_0,params_hidden_units_1,params_hidden_units_last,params_loss,params_n_layers,params_optimizer,params_use_batch_norm_0,params_use_batch_norm_1,state
0,0,0.538595,2022-07-15 18:07:44.978654,2022-07-15 18:08:40.118995,0 days 00:00:55.140341,169.0,NaN,208,mae,2,Yogi,0.0,NaN,COMPLETE
1,1,5.213333,2022-07-15 18:08:40.118995,2022-07-15 18:09:06.696598,0 days 00:00:26.577603,NaN,NaN,44,mape,1,AdamW,NaN,NaN,COMPLETE
2,2,1.860014,2022-07-15 18:09:06.697602,2022-07-15 18:09:19.100776,0 days 00:00:12.403174,209.0,NaN,51,mape,2,AdamW,0.0,NaN,COMPLETE
3,3,0.531550,2022-07-15 18:09:19.101778,2022-07-15 18:10:02.722053,0 days 00:00:43.620275,NaN,NaN,156,mae,1,Yogi,NaN,NaN,COMPLETE
4,4,0.662501,2022-07-15 18:10:02.723051,2022-07-15 18:10:24.834547,0 days 00:00:22.111496,74.0,145.0,126,mae,3,AdamW,1.0,0.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.476443,2022-07-15 19:36:39.780421,2022-07-15 19:37:49.279034,0 days 00:01:09.498613,227.0,228.0,252,mae,3,Yogi,0.0,0.0,COMPLETE
96,96,0.454185,2022-07-15 19:37:49.279034,2022-07-15 19:38:56.288019,0 days 00:01:07.008985,219.0,237.0,179,mae,3,Yogi,0.0,0.0,COMPLETE
97,97,0.456911,2022-07-15 19:38:56.289019,2022-07-15 19:40:03.373852,0 days 00:01:07.084833,237.0,189.0,218,mae,3,Yogi,0.0,0.0,COMPLETE
98,98,0.484999,2022-07-15 19:40:03.374854,2022-07-15 19:41:12.424663,0 days 00:01:09.049809,213.0,225.0,256,mape,3,Yogi,0.0,0.0,COMPLETE


# Construct Best Model

In [9]:
model = create_model(study.best_trial)
model.summary()

model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=100,
    callbacks=[early_stopping],
    validation_data=(x_test, y_test),
    verbose=1
)

Model: "sequential_110"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_404 (Dense)           (None, 240)               3120      
                                                                 
 dense_405 (Dense)           (None, 219)               52779     
                                                                 
 dense_406 (Dense)           (None, 251)               55220     
                                                                 
 dense_407 (Dense)           (None, 1)                 252       
                                                                 
Total params: 111,371
Trainable params: 111,371
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
183/183 [==============================] - 2s 5ms/step - loss: 0.9413 - mae: 0.9413 - val_loss: 0.6409 - val_mae: 0.6409
Epoch 2/100
183/183 [========================

# Generate Predictions

In [27]:
y_pred = model.predict(x_test)
predictions_df = pd.DataFrame({'Ground Truth': y_test, 'Predictions': y_pred.squeeze()})
predictions_df['Predictions'] = predictions_df['Predictions'].apply(lambda pred: round(pred))
predictions_df.to_csv('predictions.csv', index=False, sep=',')
predictions_df

21/21 [==============================] - 0s 2ms/step


,Ground Truth,Predictions
5316,6.0,6
5210,6.0,6
3518,5.0,4
1622,6.0,5
2443,5.0,5
...,...,...
6486,6.0,6
2805,6.0,7
5681,5.0,5
5771,7.0,7
